In [1]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd

In [29]:
def read_csv(csv_dir):

  print(f"\n{'='*70}")
  print("READ AND PROCESS CSV FILES")
  print(f"{'='*70}")

  # Read csv files
  csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
  print(f"Found {len(csv_files)} CSV files")

  # Read and concatenate csv files
  dataframe = [pd.read_csv(os.path.join(csv_dir, file)) for file in csv_files]
  df = pd.concat(dataframe, ignore_index=True)

  return df

In [42]:
def preprocess_bathymetry(df):
    """
    Preprocess bathymetry data by removing null depth values and adding spectral indices.

    Parameters:
    df (pd.DataFrame): Input dataframe with satellite bands and depth values

    Returns:
    pd.DataFrame: Preprocessed dataframe with spectral indices
    """

    print(f"\n{'='*70}")
    print("REMOVING NULL DEPTH VALUES")
    print(f"{'='*70}")

    initial_count = len(df)
    merged_df = df.dropna(axis=0, subset=['depth']).copy()
    removed_count = initial_count - len(merged_df)

    print(f"Removed {removed_count} rows with null depth values")
    print(f"Remaining samples: {len(merged_df)}")

    return merged_df

In [35]:
def preprocess_bathymetry(df):
    """
    Preprocess bathymetry data by removing null depth values and adding spectral indices.

    Parameters:
    df (pd.DataFrame): Input dataframe with satellite bands and depth values

    Returns:
    pd.DataFrame: Preprocessed dataframe with spectral indices
    """

    print(f"\n{'='*70}")
    print("REMOVING NULL DEPTH VALUES")
    print(f"{'='*70}")

    initial_count = len(df)
    merged_df = df.dropna(axis=0, subset=['depth']).copy()
    removed_count = initial_count - len(merged_df)

    print(f"Removed {removed_count} rows with null depth values")
    print(f"Remaining samples: {len(merged_df)}")

    print(f"\n{'='*70}")
    print("ADDING SPECTRAL INDICES")
    print(f"{'='*70}")

    # Enhanced spectral indices for bathymetry and water quality
    try:
        # 1. Standard Vegetation Indices
        merged_df['NDVI'] = (merged_df['B8'] - merged_df['B4']) / (merged_df['B8'] + merged_df['B4'])
        merged_df['NDWI'] = (merged_df['B3'] - merged_df['B8']) / (merged_df['B3'] + merged_df['B8'])
        merged_df['SAVI'] = (merged_df['B8'] - merged_df['B4']) / (merged_df['B8'] + merged_df['B4'] + 0.5) * 1.5

        # 2. Water-specific indices for bathymetry
        # Modified Normalized Difference Water Index (MNDWI)
        merged_df['MNDWI'] = (merged_df['B3'] - merged_df['B11']) / (merged_df['B3'] + merged_df['B11'])

        # Water Ratio Index
        merged_df['WRI'] = (merged_df['B3'] + merged_df['B4']) / (merged_df['B8'] + merged_df['B11'])

        # 4. Turbidity and water quality indices
        # Normalized Difference Turbidity Index
        merged_df['NDTI'] = (merged_df['B4'] - merged_df['B3']) / (merged_df['B4'] + merged_df['B3'])

        # Suspended Particulate Matter Index
        merged_df['SPMI'] = merged_df['B4'] / (merged_df['B2'] + 1e-9)

        # 5. Additional useful indices
        # Brightness Index (useful for shallow water detection)
        merged_df['BI'] = (merged_df['B2'] + merged_df['B3'] + merged_df['B4']) / 3


        print("Successfully added 10 spectral indices:")
        indices_list = ['NDVI', 'NDWI', 'SAVI', 'MNDWI', 'WRI', 'NDTI', 'SPMI', 'BI']
        for idx in indices_list:
            if idx in merged_df.columns:
                print(f"  ✓ {idx}")

        # Check for infinite values and replace with NaN
        inf_count = np.isinf(merged_df[indices_list]).sum().sum()
        if inf_count > 0:
            print(f"Replacing {inf_count} infinite values with NaN")
            merged_df = merged_df.replace([np.inf, -np.inf], np.nan)

    except KeyError as e:
        print(f"Error: Missing required band {e}")
        print("Available columns:", df.columns.tolist())
        return None
    except Exception as e:
        print(f"Error calculating spectral indices: {e}")
        return None

    print(f"\n{'='*70}")
    print("DATA QUALITY CHECK")
    print(f"{'='*70}")

    # Data quality information
    print(f"Final dataset shape: {merged_df.shape}")
    print(f"Null values in depth: {merged_df['depth'].isnull().sum()}")

    # Check spectral indices for extreme values
    spectral_cols = [col for col in merged_df.columns if col not in df.columns]
    if spectral_cols:
        print(f"\nSpectral indices value ranges:")
        for col in spectral_cols:
            if col in merged_df.columns:
                valid_data = merged_df[col].replace([np.inf, -np.inf], np.nan).dropna()
                if len(valid_data) > 0:
                    print(f"  {col}: [{valid_data.min():.3f}, {valid_data.max():.3f}]")

    return merged_df

In [40]:
def print_df(df, name="DataFrame", max_rows=20, max_cols=None, show_info=True):
    """
    Print DataFrame in a nicely formatted way.

    Parameters:
    df (pd.DataFrame): DataFrame to print
    name (str): Name to display for the DataFrame
    max_rows (int): Maximum number of rows to display
    max_cols (int): Maximum number of columns to display
    show_info (bool): Whether to show DataFrame info
    """

    print(f"\n{'='*80}")
    print(f"📊 {name.upper()}")
    print(f"{'='*80}")

    if show_info:
        print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
        print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        print(f"Columns: {list(df.columns)}")

    print(f"\n{'─'*80}")
    print("DATA PREVIEW:")
    print(f"{'─'*80}")

    # Set display options for better formatting
    with pd.option_context(
        'display.max_rows', max_rows,
        'display.max_columns', max_cols,
        'display.width', 120,
        'display.float_format', '{:.4f}'.format,
        'display.colheader_justify', 'center'
    ):
        print(df)

    print(f"{'-'*80}")

In [47]:
if __name__ == "__main__":

  dir = '/content/drive/MyDrive/Bathymetry/ECHOSOUNDER/bathymetry'

  bathy = read_csv(dir)
  bathy_processed = preprocess_bathymetry(bathy)

  print(bathy_processed)
  #print_df(bathy_processed, max_rows=10, max_cols=10)

  print(f"\n{'='*80}")
  print(f"SAVE CSV FILE")
  print(f"{'='*80}")

  out_dir = '/content/drive/MyDrive/Bathymetry/ECHOSOUNDER'
  print("saving csv file to ", out_dir)
  bathy_processed.to_csv(os.path.join(out_dir, 'bathymetry_processed.csv'), index=False)

  print("done ...")



READ AND PROCESS CSV FILES
Found 7 CSV files

REMOVING NULL DEPTH VALUES
Removed 7162 rows with null depth values
Remaining samples: 49189
          id        lat         lon  depth  wtemp      B1        B2        B3  \
0          0  15.514707  119.902056   0.45  31.32  0.0368  0.029180  0.046980   
1          1  15.514716  119.902062   0.85  31.32  0.0333  0.027885  0.043235   
2          2  15.514760  119.902070   0.78  31.32  0.0333  0.027885  0.043235   
3          3  15.514773  119.902070   0.45  31.31  0.0333  0.027885  0.043235   
4          4  15.514788  119.902071   0.76  31.31  0.0333  0.027885  0.043235   
...      ...        ...         ...    ...    ...     ...       ...       ...   
56136  24020  13.916629  120.616781   1.16  31.09  0.0493  0.045670  0.057270   
56137  24021  13.916625  120.616786   1.16  31.08  0.0493  0.045670  0.057270   
56138  24022  13.916617  120.616798   1.16  31.08  0.0493  0.045670  0.057270   
56139  24023  13.916609  120.616813   1.16  31.08 